In [2]:
import pickle
from phystables import BasicTable, SimpleTable, make_noisy
from phystables.constants import *
import define_tables
from define_tables import WallData, GoalData, BallData, TableData
import pygame as pg
import argparse
import numpy as np


DEF_NOISE = {
"kapv": KAPV_DEF, # VonMises precision on initial velocity angle estimation
"kapb": KAPB_DEF, # VonMises precision on the bounce angle estimation
"kapm": KAPM_DEF, # VonMises precision on the motion jitter
"perr": PERR_DEF  # Gaussian error on initial position estimation
}

        
def getTable(tables_file, table_id, trial_id, noisy):
    '''
    tables_file: File name with tables
    table_id: Table from table_files
    trial_id: Determines which simulation to run
    noisy: T/F
    '''
    with open(tables_file, 'rb') as input:
        tablesData = pickle.load(input)
    tableData = tablesData[table_id]
    
    table = BasicTable(dims=tableData.dims)
    for wall in tableData.walls:
        table.add_wall(upperleft=wall.upperleft, lowerright=wall.lowerright)

    for goal in tableData.goals:
        table.add_goal(upperleft=goal.upperleft, lowerright=goal.lowerright, onreturn=goal.onreturn, color=goal.color)
        
    balls = tableData.balls
    if trial_id == 0:
        table.add_ball(initpos=balls[0].initpos, initvel=balls[0].initvel, color=balls[0].color)
    elif trial_id == 1:
        table.add_ball(initpos=balls[1].initpos, initvel=balls[1].initvel, color=balls[1].color)
    else:
        table.add_ball(initpos=balls[0].initpos, initvel=balls[0].initvel, color=balls[0].color)
        table.add_ball(initpos=balls[1].initpos, initvel=balls[1].initvel, color=balls[1].color)
        
    if noisy:
        table = make_noisy(table, **DEF_NOISE)
    
    return table

def run_to_end(trial, max_time=10.):
    while True:
        r = trial.step(t=max_time, maxtime=max_time)
        if r is not None:
            if isinstance(r, list):
                r = r[0]
            return [r, trial.tm]
    
def run_simulations(tables_file, table_id, trial_id, noisy, num_simulations, max_time):    
    table = getTable(tables_file, table_id, trial_id, noisy)
    if noisy:
        table = make_noisy(table, **DEF_NOISE)
    simulations = []
    for i in range(num_simulations):
        ret, time = run_to_end(table, max_time)
        outcome = get_const(ret)
        ball = None
        sample = (table_id, outcome, time, ball)
        simulations.append(sample)
    return simulations

pygame 2.0.0 (SDL 2.0.12, python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/opt/anaconda3/lib/python3.7/site-packages/pymunk/libchipmunk.dylib]


In [3]:
def vis_trial(trial, noisy=False):
    pg.init()
    sc = pg.display.set_mode((1000,600))
    if noisy:
        trial = make_noisy(trial, **DEF_NOISE)
    trial.assign_surface(sc)
    trial.demonstrate()

In [17]:
table = getTable(tables_file='tables_metadata.pkl',
                table_id=0,
                trial_id=-1,
                noisy=True)
vis_trial(table, noisy=True)

In [4]:
!python define_tables.py

pygame 2.0.0 (SDL 2.0.12, python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/opt/anaconda3/lib/python3.7/site-packages/pymunk/libchipmunk.dylib]


# Get Samples

In [29]:
samples1 = run_simulations(tables_file='tables_metadata.pkl', 
                          table_id=0, 
                          trial_id=0, 
                          noisy=True, 
                          num_simulations=10000, 
                          max_time=10)

samples2 = run_simulations(tables_file='tables_metadata.pkl', 
                          table_id=0, 
                          trial_id=1, 
                          noisy=True, 
                          num_simulations=10000, 
                          max_time=10);

In [21]:
with open('tables_metadata.pkl', 'rb') as input:
        tablesData = pickle.load(input)
print(tablesData)

In [34]:
################
# Small Oracle #
################

import random

def small_oracle(samples, num_sim_1, num_sim_2, iters=1000):
    '''
    samples:        Pre-run samples to draw from, list [Ball_1, Ball_2]
    num_sim_1:      Number of simulations for ball 1
    num_sim_2:      Number of simulations for ball 2
    samples:        Number of samples (each sample has num_sim_1 + 2 sims)
    Return tuple(# green wins, # red wins, avg total time)
    # green wins:   Number of samples where #G > #R + 0.5 * ties
    # red wins:     Number of samples where #G < #R + 0.5 * ties
    avg total time: Average total time for a sample
    '''
    g_wins = 0  # track green wins
    r_wins = 0  # track red wins
    total_time = 0  # total time

    for _ in range(iters):  # get samples
        # wins for a fixed sample
        sample_g_wins = 0
        sample_r_wins = 0
        sample_time = 0
        # simulations for ball 1
        for sim_1 in range(num_sim_1):
            table_id, outcome, time, ball = random.choice(samples[0])
            if outcome == 'GREENGOAL':
                sample_g_wins += 1
            if outcome == 'REDGOAL':
                sample_r_wins += 1
            sample_time += time
        # simulations for ball 2
        for sim_2 in range(num_sim_2):
            table_id, outcome, time, ball = random.choice(samples[1])
            if outcome == 'GREENGOAL':
                sample_g_wins += 1
            if outcome == 'REDGOAL':
                sample_r_wins += 1
            sample_time += time
        if sample_g_wins > sample_r_wins:
            g_wins += 1
        if sample_g_wins < sample_r_wins:
            r_wins += 1
        if sample_g_wins == sample_r_wins:
            g_wins += 0.5
            r_wins += 0.5
        total_time += sample_time
    return (g_wins/iters, r_wins/iters, total_time/iters)

In [40]:
small_oracle([samples1, samples2], 10, 10, 10000)

(0.82505, 0.17495, 283.85228469997077)